In [69]:
import os

# import pkg
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Pre-process
## Investigate dataset
### Dataset

The dataset stores houshold energy usage in London between 11/2011 and 02/2014. The source is available on kaggle:  https://www.kaggle.com/jeanmidev/smart-meters-in-london

Noted that kaggle provided processed dataset for downloading, but this project starts working on the raw data files to transform and clean data.

## Read dataset
First look at data in folder `daily_dataset`


In [70]:
b0_daily = pd.read_csv('data/hhblock_dataset/block_0.csv')
b0_daily.head()

,LCLid,day,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,...,hh_38,hh_39,hh_40,hh_41,hh_42,hh_43,hh_44,hh_45,hh_46,hh_47
0,MAC000002,2012-10-13,0.263,0.269,0.275,0.256,0.211,0.136,0.161,0.119,...,0.918,0.278,0.267,0.239,0.230,0.233,0.235,0.188,0.259,0.250
1,MAC000002,2012-10-14,0.262,0.166,0.226,0.088,0.126,0.082,0.123,0.083,...,1.075,0.956,0.821,0.745,0.712,0.511,0.231,0.210,0.278,0.159
2,MAC000002,2012-10-15,0.192,0.097,0.141,0.083,0.132,0.070,0.130,0.074,...,1.164,0.249,0.225,0.258,0.260,0.334,0.299,0.236,0.241,0.237
3,MAC000002,2012-10-16,0.237,0.237,0.193,0.118,0.098,0.107,0.094,0.109,...,0.966,0.172,0.192,0.228,0.203,0.211,0.188,0.213,0.157,0.202
4,MAC000002,2012-10-17,0.157,0.211,0.155,0.169,0.101,0.117,0.084,0.118,...,0.223,0.075,0.230,0.208,0.265,0.377,0.327,0.277,0.288,0.256


### Data-type conversion

In [71]:
# check data type, noted that `day` is not parsed as date-time
b0_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25286 entries, 0 to 25285
Data columns (total 50 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   LCLid   25286 non-null  object 
 1   day     25286 non-null  object 
 2   hh_0    25286 non-null  float64
 3   hh_1    25286 non-null  float64
 4   hh_2    25286 non-null  float64
 5   hh_3    25286 non-null  float64
 6   hh_4    25286 non-null  float64
 7   hh_5    25286 non-null  float64
 8   hh_6    25286 non-null  float64
 9   hh_7    25286 non-null  float64
 10  hh_8    25286 non-null  float64
 11  hh_9    25286 non-null  float64
 12  hh_10   25286 non-null  float64
 13  hh_11   25286 non-null  float64
 14  hh_12   25286 non-null  float64
 15  hh_13   25286 non-null  float64
 16  hh_14   25286 non-null  float64
 17  hh_15   25286 non-null  float64
 18  hh_16   25286 non-null  float64
 19  hh_17   25286 non-null  float64
 20  hh_18   25286 non-null  float64
 21  hh_19   25286 non-null  float64
 22

In [72]:
# convert `day` to datetime
b0_daily.day = pd.to_datetime(b0_daily.day, format="%Y-%m-%d")

b0_daily.day.head()

0   2012-10-13
1   2012-10-14
2   2012-10-15
3   2012-10-16
4   2012-10-17
Name: day, dtype: datetime64[ns]

### Check NA for time-series

In [73]:
# look at the start date of each `LCLid` which is the user
b0_daily[['LCLid', 'day']].groupby('LCLid').min().day.value_counts()

day
2012-10-13    3
2012-09-29    3
2012-10-10    3
2012-10-02    3
2012-10-25    3
2012-10-26    2
2012-10-17    2
2012-10-03    2
2012-10-23    2
2012-10-19    2
2012-10-18    2
2012-05-24    1
2012-05-22    1
2012-07-07    1
2012-05-26    1
2012-03-17    1
2012-10-24    1
2012-03-21    1
2012-10-20    1
2012-10-12    1
2012-10-16    1
2011-12-04    1
2012-10-11    1
2012-10-09    1
2012-10-04    1
2012-12-12    1
2012-09-25    1
2012-09-22    1
2012-09-20    1
2012-09-19    1
2012-09-18    1
2012-05-09    1
2012-03-23    1
2012-04-18    1
Name: count, dtype: int64

In [74]:
# look at the end date of each `LCLid` which is the user
b0_daily[['LCLid', 'day']].groupby('LCLid').max().day.value_counts()

day
2014-02-27    42
2013-05-14     1
2012-12-20     1
2014-02-21     1
2013-08-11     1
2013-11-20     1
2013-10-15     1
2014-01-12     1
2013-09-18     1
Name: count, dtype: int64

The result shows users have different 'start date' and 'end date'.

## Transform data
### Reshape


In [76]:
# pivoting and renaming columns
b0_trans = b0_daily.set_index(['LCLid', 'day']).stack().reset_index().rename({'level_2': 'hour', 0: 'energy'}, axis=1)
b0_trans

,LCLid,day,hour,energy
0,MAC000002,2012-10-13,hh_0,0.263
1,MAC000002,2012-10-13,hh_1,0.269
2,MAC000002,2012-10-13,hh_2,0.275
3,MAC000002,2012-10-13,hh_3,0.256
4,MAC000002,2012-10-13,hh_4,0.211


In [90]:
b0_trans

,LCLid,day,hour,energy
0,MAC000002,2012-10-13,0,0.263
1,MAC000002,2012-10-13,1,0.269
2,MAC000002,2012-10-13,2,0.275
3,MAC000002,2012-10-13,3,0.256
4,MAC000002,2012-10-13,4,0.211
...,...,...,...,...
1213673,MAC005492,2014-02-27,43,0.250
1213674,MAC005492,2014-02-27,44,0.182
1213675,MAC005492,2014-02-27,45,0.122
1213676,MAC005492,2014-02-27,46,0.140


### Extract hour

In [77]:
# only keep the numeric value in column `hour`
b0_trans.hour = b0_trans.hour.str.strip().str.replace('hh_', '')
b0_trans

,LCLid,day,hour,energy
0,MAC000002,2012-10-13,0,0.263
1,MAC000002,2012-10-13,1,0.269
2,MAC000002,2012-10-13,2,0.275
3,MAC000002,2012-10-13,3,0.256
4,MAC000002,2012-10-13,4,0.211
...,...,...,...,...
1213673,MAC005492,2014-02-27,43,0.250
1213674,MAC005492,2014-02-27,44,0.182
1213675,MAC005492,2014-02-27,45,0.122
1213676,MAC005492,2014-02-27,46,0.140


### Extract time-series(for one user)
Create a dataframe which covers the earliest start date and latest end date

In [ ]:
# find the earliest start date and latest end date in one .csv file
start_date = b0_daily.day.min()
max_date = b0_daily.day.max()

# create dataframe


## Transform all data

In [88]:
# define a function to extract values from `.csv` file
def extract_ts(df):
    """

    :param df: dataframe read from '.data/hhblock_dataset/block_*.csv'
    :return:
    """

    start_date = df.day.min()
    max_date = df.day.max()
    name = df['LCLid'].unique()[0]
